In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train = pd.read_csv('Titanic_train.csv')
test=pd.read_csv('Titanic_test.csv')

In [3]:
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [4]:
av_age = train.groupby('Pclass')
av_age.mean()

,PassengerId,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,,
1,461.597222,0.629630,38.233441,0.416667,0.356481,84.154687
2,445.956522,0.472826,29.877630,0.402174,0.380435,20.662183
3,439.154786,0.242363,25.140620,0.615071,0.393075,13.675550


In [5]:
def impute_age_train(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1:
            return 38.23
        elif Pclass == 2:
            return 29.87
        else:
            return 25.14
        
    else:
        return Age

In [6]:
train['Age'] = train[['Age','Pclass']].apply(impute_age_train,axis=1)
train=train.dropna(subset = ['Embarked'])
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column)


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [7]:
av_age = test.groupby('Pclass')
av_age.mean()

,PassengerId,Age,SibSp,Parch,Fare
Pclass,,,,,
1,1098.224299,40.918367,0.476636,0.383178,94.280297
2,1117.935484,28.777500,0.376344,0.344086,22.202104
3,1094.178899,24.027945,0.463303,0.417431,12.459678


In [8]:
def impute_age_test(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1:
            return 40.9
        elif Pclass == 2:
            return 28.77
        else:
            return 24.02
        
    else:
        return Age

In [9]:
test['Age'] = test[['Age','Pclass']].apply(impute_age_test,axis=1)
missing_val_count_by_column = (test.isnull().sum())
print(missing_val_count_by_column)

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [10]:
X_train = train.drop(['PassengerId', 'Survived', 'Fare', 'Name','Ticket', 'Fare', 'Cabin'], axis=1)
y_train = train.iloc[:, 1]
X_test = test.drop(['PassengerId', 'Fare', 'Name','Ticket', 'Fare','Cabin'], axis=1)

In [11]:
X_train=pd.get_dummies(X_train, prefix=['Sex', 'Embarked'], columns=['Sex', 'Embarked'])

In [12]:
X_test=pd.get_dummies(X_test, prefix=['Sex', 'Embarked'], columns=['Sex', 'Embarked'])

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\ianki\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ianki\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\ianki\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [14]:
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [15]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
parameters = {
    "n_estimators":[1900],
    "learning_rate":[0.002],
    "max_depth":[12],
    "seed": [25],
    "reg_alpha":[0.00002,0.00006],
}
classifier = GridSearchCV(XGBClassifier(),n_jobs=-1,param_grid=parameters, scoring="roc_auc", verbose=5, cv=5)
classifier.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:   15.1s remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   16.2s remaining:   16.2s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:   16.9s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [1900], 'learning_rate': [0.002], 'max_depth': [12], 'seed': [25], 'reg_alpha': [2e-05, 6e-05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [16]:
print("Best Params: {}".format(classifier.best_params_))

Best Params: {'learning_rate': 0.002, 'max_depth': 12, 'n_estimators': 1900, 'reg_alpha': 2e-05, 'seed': 25}


In [17]:
y_pred = classifier.predict(X_test)

In [18]:
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [19]:
PassengerId = np.array(test["PassengerId"]).astype(int)
my_submission_rfc = pd.DataFrame(y_pred, PassengerId, columns = ["Survived"])
my_submission_rfc.to_csv("submission_titanicXGB.csv", index_label = ["PassengerId"])

In [20]:
f=pd.read_csv('submission_titanicXGB.csv')

In [21]:
f.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
